In [1]:
import pandas as pd
from openjij import SASampler
import traci

from functions import *

In [2]:
num_instances=10

Tall = 400
interval=20
alpha, gamma = 1, 10
yellow_signal_time, lam = 2, 2

In [3]:
result= {}

In [4]:
# 元データ
original_net_path = 'dataset/data.net.xml'
original_sumocfg_path = 'dataset/data.sumocfg'
# 最適化用に一部書き換えたデータ
new_net_path = 'dataset/modified_data.net.xml'
new_sumocfg_path = 'dataset/modified_data.sumocfg'

In [5]:
# 最適なパラメータの取得
import pickle

# データを開く
with open("best_params.pickle", "rb") as f:
    best_params = pickle.load(f)


In [6]:
def extract_params(best_params):
    R_params = {eval(k[2:]): v for k, v in best_params.items() if k.startswith('R_')}
    beta = best_params.get('beta', None)
    return R_params, beta

R_params, beta = extract_params(best_params)

In [7]:
junc_info, edge_info = get_info(new_net_path, new_sumocfg_path)
lanes_from_state = extract_lanes_from_state(new_net_path)

B = make_B(edge_info)

 Retrying in 1 seconds
***Starting server on port 50080 ***
Loading net-file from 'dataset/modified_data.net.xml' ... done (192ms).


pj_obj_create: /Users/shikanai/opt/anaconda3/envs/opt/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.
pj_obj_create: /Users/shikanai/opt/anaconda3/envs/opt/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.
pj_obj_create: /Users/shikanai/opt/anaconda3/envs/opt/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.


Loading done.
Simulation version 1.18.0 started with time: 0.00.
Traffic light 1250050006 has 2 phases.
Traffic light 1250050012 has 2 phases.
Traffic light 1250050025 has 2 phases.
Traffic light 1250050032 has 2 phases.
Traffic light 1251057179 has 2 phases.
Traffic light 1251057265 has 2 phases.
Traffic light 1251057283 has 2 phases.
Traffic light 1251057285 has 2 phases.
Traffic light 1251057371 has 3 phases.
Traffic light 1252304871 has 2 phases.
Traffic light 1252304873 has 3 phases.
Traffic light 1252304874 has 2 phases.
Traffic light 1252304875 has 2 phases.
Traffic light 1252304882 has 2 phases.
Traffic light 1252304891 has 2 phases.
Traffic light 1252304895 has 2 phases.
Traffic light 1252304897 has 2 phases.
Traffic light 1252304899 has 2 phases.
Traffic light 1252304907 has 2 phases.
Traffic light 1252304911 has 2 phases.
Traffic light 1252304915 has 2 phases.
Traffic light 1252304922 has 2 phases.
Traffic light 1252304938 has 2 phases.
Traffic light 1252304949 has 2 phases.

/Users/shikanai/Desktop/TrafficSignalOptimization/aomori_miniのコピー4/functions.py:537: UserWarning: Call to deprecated function getCompleteRedYellowGreenDefinition, use getAllProgramLogics instead.
  logic = traci.trafficlight.getCompleteRedYellowGreenDefinition(tl)


In [8]:
print('信号付き交差点の数:', len(junc_info.keys()))

total_var=0
for junc, info in junc_info.items():
    total_var += info['num_modes']
print('変数の数:', total_var)
print('Rのキーの数:', len(R_params.keys()))

信号付き交差点の数: 48
変数の数: 100
Rのキーの数: 168


<h3>シミュレーション</h3>

In [9]:
def run_simulation(
    B=None,
    R=None,
    alpha=None, beta=None, gamma=None,
    num_reads=None,
    Tall=None,
    interval=None,
    yellow_signal_time=None,
    lam=None,
    sampler=None,
    method=None,
    net_path=None,
    sumocfg_path=None,
    lanes_from_state=None,
    junc_info=None,
    is_visualize=False
    ):

    if method != 'fixed':
        BR = make_normalized_BR(B, R)

    if is_visualize:
        sumoBinary = "PATH_YOUR_SUMO-GUI"
    else:
        sumoBinary = "PATH_YOUR_SUMO"
    sumoCmd = [sumoBinary, "-c", sumocfg_path, "--no-warnings", "--log", "sumo.log"]
    traci.start(sumoCmd)

    step = -1
    yellow_junc = {}
    mode_log={}
    execution_time_log={} # [sec]
    num_car_on_map_log = {}
    vehicle_counts_log = {}
    while step < Tall:
        if step == -1:
            traci.simulationStep()
            step += 1
            continue

        vehicle_counts = count_vehicles(lanes_from_state, 1, net_path) # 右折に重み付けしない
        vehicle_counts_log[step] = vehicle_counts

        vehicle_id_list = traci.vehicle.getIDList()
        vehicle_count = len(vehicle_id_list)
        num_car_on_map_log[step] = vehicle_count
        
        # 信号切替時に黄色を挟む
        for junc, info in yellow_junc.items():
            if info['time'] == 0:
                traci.trafficlight.setRedYellowGreenState(junc, info['next_state'])
                yellow_junc[junc]['time'] -= 1
            if info['time'] > 0:
                yellow_junc[junc]['time'] -= 1

        if method == 'fixed':
            mode_log[step] = {}
            for junc in junc_info.keys():
                current_phase = traci.trafficlight.getPhase(junc)
                mode_log[step][junc] = current_phase

        if step % interval == 0 and method != 'fixed':
            # 最適化計算
            if method == 'sa':
                vehicle_counts = count_vehicles(lanes_from_state, lam, net_path) # 車の数をカウントする
                C = make_normalized_C(vehicle_counts, junc_info)
                lowest_dict, sampleset, elapsed_time = use_annealing(C, BR, alpha, beta, gamma, num_reads, sampler, junc_info)
                mode_dict = get_mode(lowest_dict)
                execution_time_log[step] =  elapsed_time # [s]
            if method == 'gurobi':
                vehicle_counts = count_vehicles(lanes_from_state, lam, net_path) # 車の数をカウントする
                C = make_normalized_C(vehicle_counts, junc_info)
                lowest_dict, runtime, elapsed_time = use_gurobi(C, BR, alpha, beta, gamma, junc_info)
                mode_dict = get_mode(lowest_dict)
                execution_time_log[step] = elapsed_time # [s]
            mode_log[step] = mode_dict


            # 信号の操作
            for junc, next_phase in mode_dict.items():
                traci.trafficlight.setProgram(junc, "0") # これが無いと挙動がおかしくなる
                current_phase = traci.trafficlight.getPhase(junc)
                current_state = traci.trafficlight.getRedYellowGreenState(junc)
                traci.trafficlight.setPhase(junc, next_phase)
                next_state = traci.trafficlight.getRedYellowGreenState(junc)
                # 切替時に黄色を挟む
                if current_phase != next_phase:
                    new_state = transition_state(current_state, next_state)
                    traci.trafficlight.setRedYellowGreenState(junc, new_state)
                    yellow_junc[junc] = {'time':yellow_signal_time, 'next_state':next_state} # 黄色を挟む時間

        traci.simulationStep()
        step += 1

    traci.close()
    total_waiting_time = calculate_total_waiting_time('dataset/tripinfo-out.xml')

    log = {'mode_log': mode_log, 'execution_time_log': execution_time_log, \
        'num_car_on_map_log': num_car_on_map_log, 'vehicle_counts_log': vehicle_counts_log}

    return total_waiting_time, log

<h3>local(beta=0, Gurobi)</h3>

In [10]:
# シミュレーションパラメータを辞書にまとめる
simulation_params = {
    'B': B, 
    'R': R_params,
    'alpha': alpha,
    'beta': 0,
    'gamma': gamma,
    'Tall': Tall,
    'interval': interval, # 最適化の感覚
    'yellow_signal_time': yellow_signal_time, # 黄色を挟む時間
    'lam': lam,  # 右折レーンの重み
    'method': 'gurobi',
    'net_path': new_net_path,
    'sumocfg_path': new_sumocfg_path,
    'lanes_from_state': lanes_from_state,
    'junc_info': junc_info,
    'is_visualize': False
}

In [11]:
result['local'] = {}

# シミュレーションを実行
total_waiting_time, log = run_simulation(**simulation_params)
print(f'Total waiting time: {total_waiting_time} [s]')
print(f'Total waiting time: {total_waiting_time/3600} [hour]')

result['local']['total_waiting_time'] = total_waiting_time
result['local']['mode_log'] = log['mode_log'] # [s]
result['local']['execution_time_log'] = log['execution_time_log'] # [s]
result['local']['num_car_on_map_log'] = log['num_car_on_map_log']

 Retrying in 1 seconds
***Starting server on port 50083 ***
Loading net-file from 'dataset/modified_data.net.xml' ... done (193ms).


pj_obj_create: /Users/shikanai/opt/anaconda3/envs/opt/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.
pj_obj_create: /Users/shikanai/opt/anaconda3/envs/opt/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.
pj_obj_create: /Users/shikanai/opt/anaconda3/envs/opt/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.


Loading done.
Simulation version 1.18.0 started with time: 0.00.
Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-07
Simulation ended at time: 401.00
Reason: TraCI requested termination.
Performance: 
 Duration: 49.47s
 TraCI-Duration: 47.31s
 Real time factor: 8.10609
 UPS: 4835.796155
Vehicles: 
 Inserted: 966
 Running: 407
 Waiting: 0
 Teleports: 7 (Jam: 2, Yield: 5)
 Emergency Stops: 10
 Emergency Braking: 13
Statistics (avg of 966):
 RouteLength: 818.42
 Speed: 4.23
 Duration: 247.64
 WaitingTime: 138.13
 TimeLoss: 203.05
 DepartDelay: 1.45
 DepartDelayWaiting: 0.00

Total waiting time: 133433.0 [s]
Total waiting time: 37.06472222222222 [hour]


<h3>Gurobi</h3>

In [12]:
# シミュレーションパラメータを辞書にまとめる
simulation_params = {
    'B': B, 
    'R': R_params,
    'alpha': alpha,
    'beta': beta,
    'gamma': gamma,
    'Tall': Tall,
    'interval': interval, # 最適化の感覚
    'yellow_signal_time': yellow_signal_time, # 黄色を挟む時間
    'lam': lam,  # 右折レーンの重み
    'method': 'gurobi',
    'net_path': new_net_path,
    'sumocfg_path': new_sumocfg_path,
    'lanes_from_state': lanes_from_state,
    'junc_info': junc_info,
    'is_visualize': False
}

In [13]:
total_waiting_time_log={}
execution_time_log={}

result['gurobi'] = {}
for t in range(num_instances):
    result['gurobi']['iter'+str(t)]={}
    # シミュレーションを実行
    total_waiting_time, log = run_simulation(**simulation_params)
    print(f'Total waiting time: {total_waiting_time} [s]')

    result['gurobi']['iter'+str(t)]['total_waiting_time'] = total_waiting_time
    result['gurobi']['iter'+str(t)]['mode_log'] = log['mode_log'] # [s]
    result['gurobi']['iter'+str(t)]['execution_time_log'] = log['execution_time_log'] # [s]
    result['gurobi']['iter'+str(t)]['num_car_on_map_log'] = log['num_car_on_map_log']

 Retrying in 1 seconds
***Starting server on port 50086 ***
Loading net-file from 'dataset/modified_data.net.xml' ... done (200ms).


pj_obj_create: /Users/shikanai/opt/anaconda3/envs/opt/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.
pj_obj_create: /Users/shikanai/opt/anaconda3/envs/opt/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.
pj_obj_create: /Users/shikanai/opt/anaconda3/envs/opt/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.


Loading done.
Simulation version 1.18.0 started with time: 0.00.
Simulation ended at time: 401.00
Reason: TraCI requested termination.
Performance: 
 Duration: 50.00s
 TraCI-Duration: 47.91s
 Real time factor: 8.01984
 UPS: 4661.066779
Vehicles: 
 Inserted: 966
 Running: 318
 Waiting: 0
 Teleports: 1 (Yield: 1)
 Emergency Stops: 4
 Emergency Braking: 9
Statistics (avg of 966):
 RouteLength: 893.16
 Speed: 4.50
 Duration: 241.26
 WaitingTime: 119.56
 TimeLoss: 193.05
 DepartDelay: 1.81
 DepartDelayWaiting: 0.00

Total waiting time: 115499.0 [s]
 Retrying in 1 seconds
***Starting server on port 50092 ***
Loading net-file from 'dataset/modified_data.net.xml' ... done (194ms).


pj_obj_create: /Users/shikanai/opt/anaconda3/envs/opt/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.
pj_obj_create: /Users/shikanai/opt/anaconda3/envs/opt/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.
pj_obj_create: /Users/shikanai/opt/anaconda3/envs/opt/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.


Loading done.
Simulation version 1.18.0 started with time: 0.00.
Simulation ended at time: 401.00
Reason: TraCI requested termination.
Performance: 
 Duration: 49.82s
 TraCI-Duration: 47.69s
 Real time factor: 8.04914
 UPS: 4678.094703
Vehicles: 
 Inserted: 966
 Running: 318
 Waiting: 0
 Teleports: 1 (Yield: 1)
 Emergency Stops: 4
 Emergency Braking: 9
Statistics (avg of 966):
 RouteLength: 893.16
 Speed: 4.50
 Duration: 241.26
 WaitingTime: 119.56
 TimeLoss: 193.05
 DepartDelay: 1.81
 DepartDelayWaiting: 0.00

Total waiting time: 115499.0 [s]
 Retrying in 1 seconds
***Starting server on port 50096 ***
Loading net-file from 'dataset/modified_data.net.xml' ... done (198ms).


...

Loading done.
Simulation version 1.18.0 started with time: 0.00.
Simulation ended at time: 401.00
Reason: TraCI requested termination.
Performance: 
 Duration: 50.03s
 TraCI-Duration: 47.94s
 Real time factor: 8.01503
 UPS: 4658.271871
Vehicles: 
 Inserted: 966
 Running: 318
 Waiting: 0
 Teleports: 1 (Yield: 1)
 Emergency Stops: 4
 Emergency Braking: 9
Statistics (avg of 966):
 RouteLength: 893.16
 Speed: 4.50
 Duration: 241.26
 WaitingTime: 119.56
 TimeLoss: 193.05
 DepartDelay: 1.81
 DepartDelayWaiting: 0.00

Total waiting time: 115499.0 [s]
 Retrying in 1 seconds
***Starting server on port 50121 ***
Loading net-file from 'dataset/modified_data.net.xml' ... done (196ms).


pj_obj_create: /Users/shikanai/opt/anaconda3/envs/opt/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.
pj_obj_create: /Users/shikanai/opt/anaconda3/envs/opt/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.
pj_obj_create: /Users/shikanai/opt/anaconda3/envs/opt/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.


Loading done.
Simulation version 1.18.0 started with time: 0.00.
Simulation ended at time: 401.00
Reason: TraCI requested termination.
Performance: 
 Duration: 50.12s
 TraCI-Duration: 48.01s
 Real time factor: 8.00128
 UPS: 4650.278349
Vehicles: 
 Inserted: 966
 Running: 318
 Waiting: 0
 Teleports: 1 (Yield: 1)
 Emergency Stops: 4
 Emergency Braking: 9
Statistics (avg of 966):
 RouteLength: 893.16
 Speed: 4.50
 Duration: 241.26
 WaitingTime: 119.56
 TimeLoss: 193.05
 DepartDelay: 1.81
 DepartDelayWaiting: 0.00

Total waiting time: 115499.0 [s]
 Retrying in 1 seconds
***Starting server on port 50125 ***
Loading net-file from 'dataset/modified_data.net.xml' ... done (189ms).


pj_obj_create: /Users/shikanai/opt/anaconda3/envs/opt/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.
pj_obj_create: /Users/shikanai/opt/anaconda3/envs/opt/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.
pj_obj_create: /Users/shikanai/opt/anaconda3/envs/opt/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.


Loading done.
Simulation version 1.18.0 started with time: 0.00.
Simulation ended at time: 401.00
Reason: TraCI requested termination.
Performance: 
 Duration: 49.80s
 TraCI-Duration: 47.72s
 Real time factor: 8.05189
 UPS: 4679.691579
Vehicles: 
 Inserted: 966
 Running: 318
 Waiting: 0
 Teleports: 1 (Yield: 1)
 Emergency Stops: 4
 Emergency Braking: 9
Statistics (avg of 966):
 RouteLength: 893.16
 Speed: 4.50
 Duration: 241.26
 WaitingTime: 119.56
 TimeLoss: 193.05
 DepartDelay: 1.81
 DepartDelayWaiting: 0.00

Total waiting time: 115499.0 [s]


<h3>SA</h3>

In [14]:
# シミュレーションパラメータを辞書にまとめる
simulation_params = {
    'B': B, 
    'R': R_params,
    'alpha': alpha,
    'beta': beta,
    'gamma': gamma,
    'num_reads': 1000,
    'Tall': Tall,
    'interval': interval, # 最適化の感覚
    'yellow_signal_time': yellow_signal_time, # 黄色を挟む時間
    'lam': lam,  # 右折レーンの重み
    'sampler': SASampler(),
    'method': 'sa',
    'net_path': new_net_path,
    'sumocfg_path': new_sumocfg_path,
    'lanes_from_state': lanes_from_state,
    'junc_info': junc_info,
    'is_visualize': False
}


In [15]:
total_waiting_time_log={}
execution_time_log={}

result['sa'] = {}
for t in range(num_instances):
    result['sa']['iter'+str(t)]={}
    # シミュレーションを実行
    total_waiting_time, log = run_simulation(**simulation_params)
    print(f'Total waiting time: {total_waiting_time} [s]')
    result['sa']['iter'+str(t)]['total_waiting_time'] = total_waiting_time
    result['sa']['iter'+str(t)]['mode_log'] = log['mode_log'] # [s]
    result['sa']['iter'+str(t)]['execution_time_log'] = log['execution_time_log'] # [s]
    result['sa']['iter'+str(t)]['num_car_on_map_log'] = log['num_car_on_map_log'] # [s]


 Retrying in 1 seconds
***Starting server on port 50139 ***
Loading net-file from 'dataset/modified_data.net.xml' ... done (197ms).


pj_obj_create: /Users/shikanai/opt/anaconda3/envs/opt/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.
pj_obj_create: /Users/shikanai/opt/anaconda3/envs/opt/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.
pj_obj_create: /Users/shikanai/opt/anaconda3/envs/opt/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.


Loading done.
Simulation version 1.18.0 started with time: 0.00.
Simulation ended at time: 401.00
Reason: TraCI requested termination.
Performance: 
 Duration: 94.06s
 TraCI-Duration: 91.92s
 Real time factor: 4.26342
 UPS: 2556.094242
Vehicles: 
 Inserted: 965 (Loaded: 966)
 Running: 389
 Waiting: 1
 Teleports: 1 (Yield: 1)
 Emergency Stops: 12
 Emergency Braking: 16
Statistics (avg of 965):
 RouteLength: 841.93
 Speed: 4.22
 Duration: 249.13
 WaitingTime: 136.22
 TimeLoss: 203.51
 DepartDelay: 1.82
 DepartDelayWaiting: 302.40

Total waiting time: 131453.0 [s]
 Retrying in 1 seconds
***Starting server on port 50142 ***
Loading net-file from 'dataset/modified_data.net.xml' ... done (198ms).


pj_obj_create: /Users/shikanai/opt/anaconda3/envs/opt/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.
pj_obj_create: /Users/shikanai/opt/anaconda3/envs/opt/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.
pj_obj_create: /Users/shikanai/opt/anaconda3/envs/opt/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.


Loading done.
Simulation version 1.18.0 started with time: 0.00.
Simulation ended at time: 401.00
Reason: TraCI requested termination.
Performance: 
 Duration: 94.07s
 TraCI-Duration: 91.94s
 Real time factor: 4.26287
 UPS: 2574.924523
Vehicles: 
 Inserted: 966
 Running: 393
 Waiting: 0
 Emergency Stops: 5
 Emergency Braking: 9
Statistics (avg of 966):
 RouteLength: 824.92
 Speed: 4.15
 Duration: 250.74
 WaitingTime: 137.26
 TimeLoss: 205.89
 DepartDelay: 2.26
 DepartDelayWaiting: 0.00

Total waiting time: 132596.0 [s]
 Retrying in 1 seconds
***Starting server on port 50147 ***
Loading net-file from 'dataset/modified_data.net.xml' ... done (196ms).


...

Loading done.
Simulation version 1.18.0 started with time: 0.00.
Simulation ended at time: 401.00
Reason: TraCI requested termination.
Performance: 
 Duration: 94.29s
 TraCI-Duration: 92.17s
 Real time factor: 4.25297
 UPS: 2548.739487
Vehicles: 
 Inserted: 966
 Running: 397
 Waiting: 0
 Teleports: 1 (Yield: 1)
 Emergency Stops: 7
 Emergency Braking: 11
Statistics (avg of 966):
 RouteLength: 843.53
 Speed: 4.25
 Duration: 248.77
 WaitingTime: 133.18
 TimeLoss: 203.01
 DepartDelay: 2.50
 DepartDelayWaiting: 0.00

Total waiting time: 128654.0 [s]
 Retrying in 1 seconds
***Starting server on port 50169 ***
Loading net-file from 'dataset/modified_data.net.xml' ... done (197ms).


pj_obj_create: /Users/shikanai/opt/anaconda3/envs/opt/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.
pj_obj_create: /Users/shikanai/opt/anaconda3/envs/opt/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.
pj_obj_create: /Users/shikanai/opt/anaconda3/envs/opt/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.


Loading done.
Simulation version 1.18.0 started with time: 0.00.
Simulation ended at time: 401.00
Reason: TraCI requested termination.
Performance: 
 Duration: 94.30s
 TraCI-Duration: 92.19s
 Real time factor: 4.2523
 UPS: 2503.000997
Vehicles: 
 Inserted: 965 (Loaded: 966)
 Running: 358
 Waiting: 1
 Teleports: 1 (Yield: 1)
 Emergency Stops: 5
 Emergency Braking: 11
Statistics (avg of 965):
 RouteLength: 859.55
 Speed: 4.40
 Duration: 244.60
 WaitingTime: 126.58
 TimeLoss: 198.07
 DepartDelay: 1.60
 DepartDelayWaiting: 302.40

Total waiting time: 122153.0 [s]
 Retrying in 1 seconds
***Starting server on port 50176 ***
Loading net-file from 'dataset/modified_data.net.xml' ... done (199ms).


pj_obj_create: /Users/shikanai/opt/anaconda3/envs/opt/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.
pj_obj_create: /Users/shikanai/opt/anaconda3/envs/opt/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.
pj_obj_create: /Users/shikanai/opt/anaconda3/envs/opt/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.


Loading done.
Simulation version 1.18.0 started with time: 0.00.
Simulation ended at time: 401.00
Reason: TraCI requested termination.
Performance: 
 Duration: 94.31s
 TraCI-Duration: 92.20s
 Real time factor: 4.25216
 UPS: 2533.428768
Vehicles: 
 Inserted: 965 (Loaded: 966)
 Running: 378
 Waiting: 1
 Teleports: 1 (Yield: 1)
 Emergency Stops: 3
 Emergency Braking: 5
Statistics (avg of 965):
 RouteLength: 839.78
 Speed: 4.31
 Duration: 247.58
 WaitingTime: 133.92
 TimeLoss: 201.98
 DepartDelay: 1.79
 DepartDelayWaiting: 302.40

Total waiting time: 129233.0 [s]


<h3>固定サイクル</h3>

In [17]:
# シミュレーションパラメータを辞書にまとめる
simulation_params = {
    'Tall': Tall,
    'interval': interval,
    'method': 'fixed',
    'net_path': original_net_path,
    'sumocfg_path': original_sumocfg_path,
    'lanes_from_state': lanes_from_state,
    'junc_info': junc_info,
    'is_visualize': False
}

In [18]:
total_waiting_time_log={}
execution_time_log={}

result['fixed'] = {}

# シミュレーションを実行
total_waiting_time, log = run_simulation(**simulation_params)
print(f'Total waiting time: {total_waiting_time} [s]')

result['fixed']['total_waiting_time'] = total_waiting_time
result['fixed']['mode_log'] = log['mode_log'] # [s]
result['fixed']['execution_time_log'] = log['execution_time_log'] # [s]
result['fixed']['num_car_on_map_log'] = log['num_car_on_map_log'] # [s]

 Retrying in 1 seconds
***Starting server on port 50186 ***
Loading net-file from 'dataset/data.net.xml' ... done (192ms).


pj_obj_create: /Users/shikanai/opt/anaconda3/envs/opt/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.
pj_obj_create: /Users/shikanai/opt/anaconda3/envs/opt/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.
pj_obj_create: /Users/shikanai/opt/anaconda3/envs/opt/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.


Loading done.
Simulation version 1.18.0 started with time: 0.00.
Simulation ended at time: 401.00
Reason: TraCI requested termination.
Performance: 
 Duration: 49.69s
 TraCI-Duration: 47.57s
 Real time factor: 8.07036
 UPS: 5083.802930
Vehicles: 
 Inserted: 966
 Running: 484
 Waiting: 0
 Teleports: 3 (Yield: 3)
Statistics (avg of 966):
 RouteLength: 770.63
 Speed: 3.76
 Duration: 261.49
 WaitingTime: 161.62
 TimeLoss: 219.29
 DepartDelay: 1.59
 DepartDelayWaiting: 0.00

Total waiting time: 156125.0 [s]


<h3>結果の保存</h3>

In [20]:
import pickle

# データを保存
with open("result.pickle", "wb") as f:
    pickle.dump(result, f)